In [1]:
# data science
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

import youtube.get_youtube_data as get_youtube_data
import machine_learning.embedding as embedding

In [2]:
# Step 1. Get the Youtube API key. The Youtube API key created in Google Cloud by following the instructions on
#         the API overview page: https://developers.google.com/youtube/v3/getting-started
YOUTUBE_API_KEY = get_youtube_data.get_youtube_api_key()

In [3]:
# Step 2. Build the Youtube client to make API calls.
youtube = get_youtube_data.make_client(YOUTUBE_API_KEY)

In [4]:
# Step 3. Perform a Youtube search with a user-specified query.
youtube_df = get_youtube_data.search_youtube(
    youtube,
    query='Patrick Bet David',
    max_vids=15,       # only grab at most 50 to reduce quota usage
    order='relevance', # default is relevance
    comments=True,
    max_comments=20,   # only grab at most 100 to reduce quota usage
    transcripts=True,
)

Searching for: Patrick Bet David
Returning 15 results


In [5]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   video_id                15 non-null     object 
 1   published_at            15 non-null     object 
 2   channel_id              15 non-null     object 
 3   title                   15 non-null     object 
 4   description             15 non-null     object 
 5   channel_title           15 non-null     object 
 6   live_broadcast_content  15 non-null     object 
 7   thumbnail_default_url   15 non-null     object 
 8   thumbnail_medium_url    15 non-null     object 
 9   thumbnail_high_url      15 non-null     object 
 10  comments                15 non-null     object 
 11  thumbnail_standard_url  15 non-null     object 
 12  thumbnail_maxres_url    15 non-null     object 
 13  tags                    15 non-null     object 
 14  video_category_id       15 non-null     obje

In [10]:
youtube_df.sample(5)

,video_id,published_at,channel_id,title,description,channel_title,live_broadcast_content,thumbnail_default_url,thumbnail_medium_url,thumbnail_high_url,...,video_category_id,video_duration,video_caption,video_view_count,video_like_count,video_comment_count,is_comments_enabled,is_live_content,NoCommentsBinary,transcript
8,SKBEsVM3xHQ,2023-05-02T23:14:34Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,Patrick Bet-David Offers Tucker Carlson $100 M...,"In this short clip, Patrick Bet-David offers T...",Valuetainment,none,https://i.ytimg.com/vi/SKBEsVM3xHQ/default.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/mqdefault.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/hqdefault.jpg,...,27,916.0,False,1119283.0,41562.0,5981.0,True,False,0,we made a public offer to Tucker Carlson on th...
9,ZaXqczhYzZo,2023-03-19T19:00:21Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,The Day I Became A NEW MAN - Emotional Story b...,shorts #short #valuetainment #patrickbetdavid.,Valuetainment,none,https://i.ytimg.com/vi/ZaXqczhYzZo/default.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/mqdefault.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/hqdefault.jpg,...,27,53.0,False,281423.0,13257.0,157.0,True,False,0,and I went to UCLA Medical Center and they wer...
13,UZAYgSUPF6Q,2023-06-14T02:00:16Z,UCAjmXPKv1zpYftSDAMeJz8A,Patrick Bet-David Explains Who George Soros is...,Watch Full Episode - https://youtu.be/TGx_HUPw...,FLAGRANT CLIPS,none,https://i.ytimg.com/vi/UZAYgSUPF6Q/default.jpg,https://i.ytimg.com/vi/UZAYgSUPF6Q/mqdefault.jpg,https://i.ytimg.com/vi/UZAYgSUPF6Q/hqdefault.jpg,...,23,978.0,False,949902.0,17271.0,2952.0,True,False,0,is a patriot no okay I don't know what that gu...
2,r_6jIaB_Kzk,2023-07-21T20:00:14Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,PBD DESTROYS Anthony Weiner,shorts PBD DESTROYS Anthony Weiner.,Valuetainment,none,https://i.ytimg.com/vi/r_6jIaB_Kzk/default.jpg,https://i.ytimg.com/vi/r_6jIaB_Kzk/mqdefault.jpg,https://i.ytimg.com/vi/r_6jIaB_Kzk/hqdefault.jpg,...,24,53.0,False,123962.0,7081.0,767.0,True,False,0,this is who you are you walk up here and you s...
12,4XL4P2IZLxo,2023-07-05T13:00:07Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,LGBTQ Movement Exposed: Shocking History Behin...,"In this episode, Patrick exposes the shocking ...",Valuetainment,none,https://i.ytimg.com/vi/4XL4P2IZLxo/default.jpg,https://i.ytimg.com/vi/4XL4P2IZLxo/mqdefault.jpg,https://i.ytimg.com/vi/4XL4P2IZLxo/hqdefault.jpg,...,24,1354.0,False,1044376.0,83451.0,11688.0,True,False,0,what have you learned the last few years of ce...


In [7]:
filter_sent = "Politics"
list_of_videos = ["Who'se Really Supporting Russia",
                  "The Perfect Hillary Clinton Analogy",
                  "The Evolution of Alex Jones",\
                  "Patrick Bet David on The Breakfast Club",
                  "The Truth About The 2020 Election",
                  "Kobe Bryant's Last Great Interview"]
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding.filter_out_embed(model,filter_sent,list_of_videos)


['Patrick Bet David on The Breakfast Club',
 "Kobe Bryant's Last Great Interview"]

In [8]:
titles = youtube_df['title'].tolist()
print("titles:",titles)
print("api test")
print(embedding.filter_out_embed(model,filter_sent,titles))

titles: ['UNHINGED: Anthony Weiner Has a Meltdown When Asked About Clinton’s Kill List Allegation', 'Dave Smith | Live PBD Podcast | Ep. 288', 'PBD DESTROYS Anthony Weiner', 'Anthony Weiner MELTDOWN | PBD Podcast | Ep. 287', 'Chris Cuomo Uncensored | PBD Podcast | Ep. 289', 'Why Hollywood Elites Don’t Want You To Watch Sound Of Freedom', 'Are Muslims The Angriest People In The World?', '“He’s a Conspiracy Charlatan!” - Anthony Weiner SLAMS RFK', 'Patrick Bet-David Offers Tucker Carlson $100 Million To Join Valuetainment', 'The Day I Became A NEW MAN - Emotional Story by Patrick Bet-David', 'Inside Patrick Bet-David&#39;s $25 Million Dream Home', '&quot;Why Would I Drink?&quot; - Patrick Bet-David Expains Why He Doesn&#39;t Drink Often', 'LGBTQ Movement Exposed: Shocking History Behind It', 'Patrick Bet-David Explains Who George Soros is &amp; Why He&#39;s Important in Society', 'Patrick Bet-David Makes Tucker Carlson $100 Million Offer to Join His@VALUETAINMENT Content Company']
api te